In [1]:
import pandas as pd
df=pd.read_csv(r"C:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Raw_data\faers_25Q4_targets_multilabel_v2.csv")

C:\Users\xolmu\AppData\Local\Temp\ipykernel_37836\3168271097.py:2: DtypeWarning: Columns (0: to_mfr, 1: RPSR_rpsr_cod) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Raw_data\faers_25Q4_targets_multilabel_v2.csv")


In [2]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 308959 entries, 0 to 308958
Data columns (total 75 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   primaryid                   308959 non-null  int64  
 1   caseid                      308959 non-null  int64  
 2   caseversion                 308959 non-null  int64  
 3   i_f_code                    308959 non-null  str    
 4   event_dt                    148733 non-null  float64
 5   mfr_dt                      308959 non-null  int64  
 6   init_fda_dt                 308959 non-null  int64  
 7   fda_dt                      308959 non-null  int64  
 8   rept_cod                    308959 non-null  str    
 9   auth_num                    32133 non-null   str    
 10  mfr_num                     300199 non-null  str    
 11  mfr_sndr                    308959 non-null  str    
 12  lit_ref                     28486 non-null   str    
 13  age                      

In [3]:
df.columns

Index(['primaryid', 'caseid', 'caseversion', 'i_f_code', 'event_dt', 'mfr_dt',
       'init_fda_dt', 'fda_dt', 'rept_cod', 'auth_num', 'mfr_num', 'mfr_sndr',
       'lit_ref', 'age', 'age_cod', 'age_grp', 'sex', 'e_sub', 'wt', 'wt_cod',
       'rept_dt', 'to_mfr', 'occp_cod', 'reporter_country', 'occr_country',
       'DRUG_n_rows', 'DRUG_drugname', 'DRUG_prod_ai', 'DRUG_role_cod',
       'DRUG_route', 'DRUG_dose_amt', 'DRUG_dose_unit', 'DRUG_dose_freq',
       'INDI_n_rows', 'INDI_indi_pt', 'OUTC_n_rows', 'OUTC_outc_cod',
       'REAC_n_rows', 'REAC_pt', 'RPSR_n_rows', 'RPSR_rpsr_cod', 'THER_n_rows',
       'THER_start_dt', 'THER_end_dt', 'THER_dur', 'THER_dur_cod',
       'THER_dsg_drug_seq', 'REAC_pt_symptom', 'REAC_pt_dropped',
       'REAC_n_rows_symptom', 'REAC_pt_symptom_v2', 'REAC_pt_dropped_v2',
       'REAC_n_rows_symptom_v2', 'y_gastrointestinal', 'y_cardiovascular',
       'y_neurological', 'y_dermatologic', 'y_hematologic', 'y_hepatic',
       'y_renal', 'y_respiratory', '

In [4]:
df.isnull().sum()

primaryid                          0
caseid                             0
caseversion                        0
i_f_code                           0
event_dt                      160226
                               ...  
y_metabolic_endocrine              0
y_hypersensitivity_allergy         0
y_injury_accident                  0
y_pregnancy_reproductive           0
y_labels                       59209
Length: 75, dtype: int64

In [5]:
# “Real missing”ni NaN + bo‘sh string + tokenlar bilan birga sanash (eng to‘g‘ri diagnostika)

# Mana bitta universal diagnostika:

import pandas as pd
import numpy as np

bad_tokens = {"", "nan", "none", "null", "n/a", "na"}

def missing_like_count(col: pd.Series) -> int:
    if col.dtype == "O":
        s = col.astype(str).str.strip().str.lower()
        return int(col.isna().sum() + s.isin(bad_tokens).sum())
    return int(col.isna().sum())

miss_like = df.apply(missing_like_count).sort_values(ascending=False)
miss_like.head(25)

# Bu sizga isnull() ko‘rmayotgan “missing-like” holatlarni ham chiqarib beradi.

RPSR_rpsr_cod         301494
RPSR_n_rows           301494
to_mfr                300445
lit_ref               280473
auth_num              276826
REAC_pt_dropped_v2    274368
THER_dur              256721
THER_dur_cod          256715
wt_cod                249502
wt                    249502
THER_end_dt           235695
REAC_pt_dropped       221856
age_grp               192564
DRUG_dose_freq        176186
event_dt              160226
THER_start_dt         158433
THER_dsg_drug_seq     147854
THER_n_rows           147854
DRUG_dose_amt         115186
DRUG_dose_unit        115186
OUTC_outc_cod         113343
OUTC_n_rows           113343
age                   107079
age_cod               107067
DRUG_route             61769
dtype: int64

In [6]:
import numpy as np
import pandas as pd
import re

obj_cols = df.select_dtypes(include=["object"]).columns

def to_nan_tokens(s: pd.Series) -> pd.Series:
    x = s.astype(str).str.strip()
    # bo'sh -> NaN
    x = x.replace(r"^\s*$", np.nan, regex=True)
    # tokenlar -> NaN (case-insensitive)
    x = x.replace(r"^(?i:nan|null|none|n/a|na)$", np.nan, regex=True)
    return x

df[obj_cols] = df[obj_cols].apply(to_nan_tokens)

df.isnull().sum().sort_values(ascending=False).head(25)

C:\Users\xolmu\AppData\Local\Temp\ipykernel_41244\3220003273.py:5: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  obj_cols = df.select_dtypes(include=["object"]).columns


RPSR_rpsr_cod         301494
RPSR_n_rows           301494
to_mfr                300445
lit_ref               280473
auth_num              276826
REAC_pt_dropped_v2    274368
THER_dur              256721
THER_dur_cod          256715
wt_cod                249502
wt                    249502
THER_end_dt           235695
REAC_pt_dropped       221856
age_grp               192564
DRUG_dose_freq        176186
event_dt              160226
THER_start_dt         158433
THER_dsg_drug_seq     147854
THER_n_rows           147854
DRUG_dose_amt         115186
DRUG_dose_unit        115186
OUTC_outc_cod         113343
OUTC_n_rows           113343
age                   107079
age_cod               107067
DRUG_route             61769
dtype: int64

In [6]:
df.isnull().sum()

primaryid                          0
caseid                             0
caseversion                        0
i_f_code                           0
event_dt                      160226
                               ...  
y_metabolic_endocrine              0
y_hypersensitivity_allergy         0
y_injury_accident                  0
y_pregnancy_reproductive           0
y_labels                       59209
Length: 75, dtype: int64

In [7]:
#CELL 1 — Import + Path + Load
from pathlib import Path
import pandas as pd
import numpy as np

# Notebook qayerda turganiga qarab project root
CWD = Path.cwd()
if (CWD / "Data").exists():
    PROJECT_ROOT = CWD
elif (CWD.parent / "Data").exists():
    PROJECT_ROOT = CWD.parent
else:
    PROJECT_ROOT = CWD

IN_CSV = PROJECT_ROOT / "Data" / "Raw_data" / "faers_25Q4_targets_multilabel_v2.csv"
OUT_DIR = PROJECT_ROOT / "Data" / "Processed" / "splits_multilabel_noleakage"
OUT_DIR.mkdir(parents=True, exist_ok=True)
print("OUT_DIR:", OUT_DIR)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("IN_CSV exists:", IN_CSV.exists(), IN_CSV)
print("OUT_DIR:", OUT_DIR)

df = pd.read_csv(IN_CSV, low_memory=False)
print("Loaded:", df.shape)
df.head(2)

OUT_DIR: c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\splits_multilabel_noleakage
PROJECT_ROOT: c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract
IN_CSV exists: True c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Raw_data\faers_25Q4_targets_multilabel_v2.csv
OUT_DIR: c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\splits_multilabel_noleakage
Loaded: (308959, 75)


,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,y_infections,y_psychiatric,y_urinary,y_injection_site,y_ocular_visual,y_metabolic_endocrine,y_hypersensitivity_allergy,y_injury_accident,y_pregnancy_reproductive,y_labels
0,100324053,10032405,3,F,NaN,20251110,20140320,20251120,EXP,NaN,...,0,0,0,0,0,0,0,0,0,NaN
1,1012809821,10128098,21,F,20140219.0,20251003,20140428,20251010,EXP,NaN,...,1,0,0,1,0,1,1,0,0,cardiovascular; respiratory; general_systemic;...


In [9]:
#CELL 2
import numpy as np
import pandas as pd

# 1) Target ustunlari: y_ bilan boshlanadi, lekin y_labels emas
y_cols = sorted([c for c in df.columns if c.startswith("y_") and c != "y_labels"])

TEXT_COL = "REAC_pt_symptom_v2"
if TEXT_COL not in df.columns:
    raise ValueError(f"Input text ustuni topilmadi: {TEXT_COL}")

print("Num labels:", len(y_cols))
print("Labels:", y_cols)

# --- Text tozalash ---
df[TEXT_COL] = df[TEXT_COL].fillna("").astype(str).str.strip()
print("Empty text rows:", int((df[TEXT_COL] == "").sum()))

# --- Y ni 0/1 ga majburan tozalash (xavfsiz) ---
for c in y_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")  # '0'/'1' string bo'lsa -> intga o‘tadi

df[y_cols] = df[y_cols].fillna(0).astype(int)

Y = df[y_cols].copy()

# --- Label sanity (np.unique o‘rniga aralash turdan himoyalangan usul) ---
vals = set(pd.unique(Y.to_numpy().ravel()))
print("Unique values in Y:", vals)
if not vals.issubset({0, 1}):
    bad = sorted([v for v in vals if v not in {0, 1}])
    raise ValueError(f"Y ichida 0/1 dan boshqa qiymat bor: {bad}")

label_count = Y.sum(axis=1)
print("Min labels per row:", int(label_count.min()))
print("Avg labels per row:", float(label_count.mean().round(3)))
print("Max labels per row:", int(label_count.max()))

bad0 = int((label_count == 0).sum())
print("Rows with 0 labels:", bad0)
if bad0 > 0:
    df = df[label_count > 0].copy()
    Y = df[y_cols].copy()
    print("After dropping 0-label rows:", df.shape)
    
    print("After cleaning:", df.shape)
    df.head(2)

Num labels: 21
Labels: ['y_cardiovascular', 'y_dermatologic', 'y_edema_swelling', 'y_gastrointestinal', 'y_general_systemic', 'y_hematologic', 'y_hepatic', 'y_hypersensitivity_allergy', 'y_infections', 'y_injection_site', 'y_injury_accident', 'y_metabolic_endocrine', 'y_musculoskeletal', 'y_neurological', 'y_ocular_visual', 'y_pain_general', 'y_pregnancy_reproductive', 'y_psychiatric', 'y_renal', 'y_respiratory', 'y_urinary']
Empty text rows: 0
Unique values in Y: {np.int64(0), np.int64(1)}
Min labels per row: 1
Avg labels per row: 2.132
Max labels per row: 18
Rows with 0 labels: 0


In [10]:
# CELL 3 — Label prevalence (umumiy balans)
preval = (Y.mean() * 100).sort_values(ascending=False).round(2)
preval_df = pd.DataFrame({"label": preval.index, "percent_of_reports": preval.values})
preval_df

,label,percent_of_reports
0,y_gastrointestinal,21.53
1,y_pain_general,19.38
2,y_dermatologic,18.78
3,y_neurological,17.67
4,y_general_systemic,16.89
5,y_infections,14.81
6,y_respiratory,12.49
7,y_cardiovascular,11.26
8,y_hematologic,10.46
9,y_musculoskeletal,9.41


In [11]:
# CELL 4 — Multi-label stratified split (agar mavjud bo‘lsa), bo‘lmasa fallback random split

# Bu cell sizga train/val/test beradi.

# Test = 10%

# Val = 10% (train ichidan)

# CELL 4 — NO-LEAKAGE Multi-label split (hash-grouped)

import numpy as np
from pandas.util import hash_pandas_object

SEED = 42
TEST_SIZE = 0.10
VAL_SIZE  = 0.10  # train ichidan

# 1) text_hash qo‘shamiz (exact text group)
df = df.copy()
df["text_hash"] = hash_pandas_object(df[TEXT_COL].fillna("").astype(str), index=False)

# 2) Unique text dataset (har hash 1 marta)
df_u = df.drop_duplicates(subset=["text_hash"]).reset_index(drop=True)
X_u = df_u[TEXT_COL].values
Y_u = df_u[y_cols].values

def multilabel_split_idx(X, Y, test_size=0.1, seed=42):
    try:
        from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
        sss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
        tr_idx, te_idx = next(sss.split(X, Y))
        return tr_idx, te_idx, "multilabel_stratified"
    except Exception:
        rng = np.random.default_rng(seed)
        idx = np.arange(len(X))
        rng.shuffle(idx)
        n_te = int(len(X) * test_size)
        te_idx = idx[:n_te]
        tr_idx = idx[n_te:]
        return tr_idx, te_idx, "random_fallback"

# 3) train vs test (unique-hash level)
tr_u_idx, te_u_idx, m1 = multilabel_split_idx(X_u, Y_u, TEST_SIZE, SEED)

# 4) train -> val (unique-hash level)
X_tr_u = X_u[tr_u_idx]
Y_tr_u = Y_u[tr_u_idx]
tr2_u_idx, va_rel_u_idx, m2 = multilabel_split_idx(X_tr_u, Y_tr_u, VAL_SIZE, SEED + 1)

train_hashes = set(df_u.iloc[tr_u_idx[tr2_u_idx]]["text_hash"].values)
val_hashes   = set(df_u.iloc[tr_u_idx[va_rel_u_idx]]["text_hash"].values)
test_hashes  = set(df_u.iloc[te_u_idx]["text_hash"].values)

print("Methods:", m1, "/", m2)
print("Unique texts:", len(df_u))
print("Unique hashes sizes:", len(train_hashes), len(val_hashes), len(test_hashes))

# 5) Full df’ni hash bo‘yicha split qilamiz (exact leakage = 0)
df_train = df[df["text_hash"].isin(train_hashes)].copy()
df_val   = df[df["text_hash"].isin(val_hashes)].copy()
df_test  = df[df["text_hash"].isin(test_hashes)].copy()

print("Shapes:", df_train.shape, df_val.shape, df_test.shape)

# 6) Verify leakage = 0
htr = set(df_train["text_hash"].values)
hva = set(df_val["text_hash"].values)
hte = set(df_test["text_hash"].values)

print("Leak train∩val :", len(htr & hva))
print("Leak train∩test:", len(htr & hte))
print("Leak val∩test  :", len(hva & hte))

Methods: multilabel_stratified / multilabel_stratified
Unique texts: 142046
Unique hashes sizes: 115317 12581 14148
Shapes: (201176, 76) (24410, 76) (24164, 76)
Leak train∩val : 0
Leak train∩test: 0
Leak val∩test  : 0


In [12]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
print(MultilabelStratifiedShuffleSplit)

<class 'iterstrat.ml_stratifiers.MultilabelStratifiedShuffleSplit'>


In [13]:
# CELL 5 — Split balansini tekshirish (train/val/test)
def show_split_stats(name, dfx):
    Yx = dfx[y_cols]
    prev = (Yx.mean() * 100).round(2)
    density = float(Yx.sum(axis=1).mean().round(3))
    print(f"\n{name}: rows={len(dfx):,} | avg_labels_per_row={density}")
    print(prev.sort_values(ascending=False).head(10))  # top10 label

show_split_stats("TRAIN", df_train)
show_split_stats("VAL  ", df_val)
show_split_stats("TEST ", df_test)


TRAIN: rows=201,176 | avg_labels_per_row=2.125
y_gastrointestinal    21.84
y_dermatologic        18.42
y_pain_general        18.22
y_neurological        17.96
y_general_systemic    17.28
y_infections          15.14
y_respiratory         12.84
y_cardiovascular      11.21
y_hematologic         10.15
y_musculoskeletal      9.75
dtype: float64

VAL  : rows=24,410 | avg_labels_per_row=2.173
y_pain_general        30.90
y_injection_site      20.86
y_gastrointestinal    20.12
y_dermatologic        18.33
y_neurological        16.01
y_general_systemic    13.74
y_infections          13.61
y_respiratory         10.41
y_cardiovascular      10.36
y_psychiatric          9.49
dtype: float64

TEST : rows=24,164 | avg_labels_per_row=2.147
y_dermatologic        22.24
y_gastrointestinal    20.33
y_pain_general        17.36
y_neurological        16.90
y_general_systemic    16.87
y_hematologic         14.09
y_infections          13.26
y_cardiovascular      12.56
y_respiratory         11.65
y_musculoskeleta

In [14]:
# CELL 6 — Saqlash (Data/Processed/ ichiga)

# Bu yerda minimal kerakli ustunlarni saqlaymiz:

# primaryid (bo‘lsa)

# REAC_pt_symptom (input text)

# y_* label ustunlari

# y_labels (debug uchun, bo‘lsa)

# CELL 6 — Save (NO-LEAK)

# CELL 6 — Save splits + ALL_CLEAN + META (NO-LEAK V2)

import json
from pathlib import Path
import numpy as np

# --- minimal kerakli ustunlar ---
keep_cols = []
if "primaryid" in df.columns:
    keep_cols.append("primaryid")

keep_cols += [TEXT_COL] + y_cols

if "y_labels" in df.columns:
    keep_cols.append("y_labels")

# --- paths ---
train_path = OUT_DIR / "train.csv"
val_path   = OUT_DIR / "val.csv"
test_path  = OUT_DIR / "test.csv"
all_path   = OUT_DIR / "all_clean.csv"
meta_path  = OUT_DIR / "split_meta.json"

# --- save split files ---
df_train[keep_cols].to_csv(train_path, index=False, encoding="utf-8-sig")
df_val[keep_cols].to_csv(val_path, index=False, encoding="utf-8-sig")
df_test[keep_cols].to_csv(test_path, index=False, encoding="utf-8-sig")

# --- save ALL_CLEAN (train+val+test) ---
df_all = pd.concat([df_train[keep_cols], df_val[keep_cols], df_test[keep_cols]], axis=0, ignore_index=True)

# (ixtiyoriy) primaryid bo‘lsa duplicates bo‘lmasligi kerak, lekin tekshirib qo‘yamiz
if "primaryid" in df_all.columns:
    before = len(df_all)
    df_all = df_all.drop_duplicates(subset=["primaryid"]).copy()
    after = len(df_all)
    print("ALL_CLEAN drop_duplicates(primaryid):", before, "->", after)

df_all.to_csv(all_path, index=False, encoding="utf-8-sig")

# --- META: “bu ishlar bo‘lganmi yo‘qmi”ni isbotlab turadi ---
# 1) leakage verify (hash orqali)
#    NOTE: df_train/df_val/df_test ichida text_hash ustuni bor (CELL 4 dan)
htr = set(df_train["text_hash"].values) if "text_hash" in df_train.columns else set()
hva = set(df_val["text_hash"].values) if "text_hash" in df_val.columns else set()
hte = set(df_test["text_hash"].values) if "text_hash" in df_test.columns else set()

meta = {
    "text_col": TEXT_COL,
    "num_labels": len(y_cols),
    "labels": y_cols,  # xohlasangiz olib tashlang (katta bo‘lmasin desangiz)
    "rows_train": int(len(df_train)),
    "rows_val": int(len(df_val)),
    "rows_test": int(len(df_test)),
    "rows_all_clean": int(len(df_all)),
    "noleak_hash_overlap_train_val": int(len(htr & hva)) if htr and hva else None,
    "noleak_hash_overlap_train_test": int(len(htr & hte)) if htr and hte else None,
    "noleak_hash_overlap_val_test": int(len(hva & hte)) if hva and hte else None,
    "note": "If overlaps are 0, NO-LEAK hash-group split was applied.",
}

# 2) 0-label drop isboti: df_all’da 0-label yo‘q bo‘lishi kerak
row_sum = df_all[y_cols].sum(axis=1)
meta["zero_label_rows_in_all_clean"] = int((row_sum == 0).sum())

# 3) Text empty yo‘qligi
meta["empty_text_rows_in_all_clean"] = int((df_all[TEXT_COL].fillna("").astype(str).str.strip() == "").sum())

meta_path.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")

print("Saved NO-LEAK V2:")
print(" -", train_path)
print(" -", val_path)
print(" -", test_path)
print(" -", all_path)
print(" -", meta_path)

# 03 bosqich natijasi nima?

# bizda endi tayyor:

# Data/Processed/splits_multilabel/train.csv

# Data/Processed/splits_multilabel/val.csv

# Data/Processed/splits_multilabel/test.csv

ALL_CLEAN drop_duplicates(primaryid): 249750 -> 249750
Saved NO-LEAK V2:
 - c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\splits_multilabel_noleakage\train.csv
 - c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\splits_multilabel_noleakage\val.csv
 - c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\splits_multilabel_noleakage\test.csv
 - c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\splits_multilabel_noleakage\all_clean.csv
 - c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\splits_multilabel_noleakage\split_meta.json


In [16]:
# o‘zi train/val/test/all_clean joylashgan papkani topadi va o‘qiydi.

#  Auto-path loader (eng oson)

from pathlib import Path
import pandas as pd
from pandas.util import hash_pandas_object

# 1) project root topish (Data/Results/visuals bo'yicha)
CWD = Path.cwd()
if (CWD / "Data").exists():
    ROOT = CWD
elif (CWD.parent / "Data").exists():
    ROOT = CWD.parent
else:
    ROOT = CWD

# 2) o'zimiz saqlagan papka nomini shu yerga yozamiz (aniq qanday bo'lsa)
# masalan: "splits_multilabel_noleak_v2" yoki "splits_multilabel_noleakage2"
SPLIT_DIR = ROOT / "Data" / "Processed" / "splits_multilabel_noleakage"

print("ROOT:", ROOT)
print("SPLIT_DIR exists:", SPLIT_DIR.exists(), SPLIT_DIR)

# 3) fayllar borligini ko'rsatib beradi
print("Files:", [p.name for p in SPLIT_DIR.glob("*.csv")])

train = pd.read_csv(SPLIT_DIR / "train.csv")
val   = pd.read_csv(SPLIT_DIR / "val.csv")
test  = pd.read_csv(SPLIT_DIR / "test.csv")
allc  = pd.read_csv(SPLIT_DIR / "all_clean.csv")

TEXT_COL = "REAC_pt_symptom_v2"
y_cols = [c for c in allc.columns if c.startswith("y_") and c != "y_labels"]

print("Shapes:", train.shape, val.shape, test.shape, allc.shape)
print("0-label rows in all_clean:", int((allc[y_cols].sum(axis=1)==0).sum()))
print("empty text in all_clean:", int((allc[TEXT_COL].fillna('').astype(str).str.strip()=='').sum()))

htr = set(hash_pandas_object(train[TEXT_COL].fillna(""), index=False))
hva = set(hash_pandas_object(val[TEXT_COL].fillna(""), index=False))
hte = set(hash_pandas_object(test[TEXT_COL].fillna(""), index=False))

print("Leak train∩val :", len(htr & hva))
print("Leak train∩test:", len(htr & hte))
print("Leak val∩test  :", len(hva & hte))

ROOT: c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract
SPLIT_DIR exists: True c:\Users\xolmu\OneDrive\Desktop\Modul Program oyi\Modul_Program3\6_project_dori_tasiri_extract\Data\Processed\splits_multilabel_noleakage
Files: ['all_clean.csv', 'test.csv', 'train.csv', 'val.csv']
Shapes: (201176, 24) (24410, 24) (24164, 24) (249750, 24)
0-label rows in all_clean: 0
empty text in all_clean: 0
Leak train∩val : 0
Leak train∩test: 0
Leak val∩test  : 0
